# Day 3 - Morning Session Exercises
## Error Handling, Testing, and Debugging

**Instructions:**
- Complete exercises appropriate to your skill level
- Experiment and modify the code
- Ask questions if you get stuck!
- Solutions are hidden below each exercise - try to solve them first!

---

## Exercise 3.1: Error Handling (35 min)

### Physics Context
Real-world data analysis pipelines encounter many issues: missing files, corrupted data, unphysical values. Robust error handling keeps your analysis running smoothly.

In [ ]:
import numpy as np
import pandas as pd
import os
%matplotlib inline

### Beginner Version: Safe Data Loading and Validation

In [ ]:
# TODO: Write a function that safely loads data from a CSV file
# Handle: FileNotFoundError, empty files, and general exceptions

def safe_load_data(filepath):
    """
    Safely load data from a CSV file.
    
    Parameters:
    -----------
    filepath : str
        Path to the CSV file
    
    Returns:
    --------
    pd.DataFrame or None : Loaded data, or None if loading failed
    """
    # YOUR CODE HERE
    # Use try/except to handle:
    # 1. FileNotFoundError - file doesn't exist
    # 2. pd.errors.EmptyDataError - file is empty
    # 3. Exception - any other error
    
    pass

# Test with non-existent file
result = safe_load_data('nonexistent_file.csv')
print(f"Result for missing file: {result}")

# Test with a valid path (if you have data)
# result = safe_load_data('data/events.csv')

<details>
<summary>💡 Click to reveal solution</summary>

```python
def safe_load_data(filepath):
    """
    Safely load data from a CSV file.
    """
    try:
        df = pd.read_csv(filepath)
        print(f"✓ Successfully loaded {len(df)} rows from {filepath}")
        return df
    except FileNotFoundError:
        print(f"✗ Error: File '{filepath}' not found")
        return None
    except pd.errors.EmptyDataError:
        print(f"✗ Error: File '{filepath}' is empty")
        return None
    except Exception as e:
        print(f"✗ Unexpected error loading {filepath}: {e}")
        return None
```

</details>

In [ ]:
# TODO: Write a function to validate energy values
# Energy must be: positive, within detector range (0-500 GeV), and not NaN

def validate_energy(energies, min_energy=0, max_energy=500):
    """
    Validate energy values and return cleaned array.
    
    Parameters:
    -----------
    energies : array-like
        Energy values to validate
    min_energy : float
        Minimum valid energy (default: 0)
    max_energy : float
        Maximum valid energy (default: 500)
    
    Returns:
    --------
    np.ndarray : Validated energies (invalid values replaced with NaN)
    dict : Validation report
    """
    energies = np.asarray(energies, dtype=float)
    
    # YOUR CODE HERE
    # Track issues and replace invalid values with NaN
    # Count: n_negative, n_out_of_range, n_nan
    
    report = {
        'total': len(energies),
        'n_negative': None,
        'n_out_of_range': None,
        'n_nan': None,
        'n_valid': None
    }
    
    return energies, report

# Test with problematic data
test_energies = [50, -10, 30, 600, np.nan, 100, -5, 200]
validated, report = validate_energy(test_energies)

print("Validation Report:")
for key, value in report.items():
    print(f"  {key}: {value}")
print(f"\nValidated energies: {validated}")

<details>
<summary>💡 Click to reveal solution</summary>

```python
def validate_energy(energies, min_energy=0, max_energy=500):
    """
    Validate energy values and return cleaned array.
    """
    energies = np.asarray(energies, dtype=float)
    
    # Count issues
    n_nan_original = np.sum(np.isnan(energies))
    n_negative = np.sum(energies < 0)
    n_too_high = np.sum(energies > max_energy)
    
    # Create cleaned array
    cleaned = energies.copy()
    
    # Replace invalid values with NaN
    cleaned[cleaned < min_energy] = np.nan
    cleaned[cleaned > max_energy] = np.nan
    
    n_valid = np.sum(~np.isnan(cleaned))
    
    report = {
        'total': len(energies),
        'n_negative': n_negative,
        'n_out_of_range': n_too_high,
        'n_nan': n_nan_original,
        'n_valid': n_valid
    }
    
    # Print warnings
    if n_negative > 0:
        print(f"⚠ Warning: {n_negative} negative energy values found")
    if n_too_high > 0:
        print(f"⚠ Warning: {n_too_high} values exceed {max_energy} GeV")
    
    return cleaned, report
```

</details>

In [ ]:
# TODO: Write a function to validate particle kinematics
# Check: pT >= 0, |eta| < 5 (reasonable for detectors), phi in [-π, π]

def validate_kinematics(pt, eta, phi):
    """
    Validate particle kinematic values.
    
    Parameters:
    -----------
    pt, eta, phi : array-like
        Kinematic variables
    
    Returns:
    --------
    bool : True if all valid
    list : List of error messages (empty if valid)
    """
    errors = []
    
    pt = np.asarray(pt)
    eta = np.asarray(eta)
    phi = np.asarray(phi)
    
    # YOUR CODE HERE
    # Check each condition and add error messages to the list
    
    return len(errors) == 0, errors

# Test with valid data
valid, errors = validate_kinematics(
    pt=[30, 45, 25],
    eta=[0.5, -1.2, 2.0],
    phi=[0.1, -2.5, 1.5]
)
print(f"Valid data: {valid}, Errors: {errors}")

# Test with invalid data
valid, errors = validate_kinematics(
    pt=[-10, 45, 25],
    eta=[0.5, -6.0, 2.0],
    phi=[0.1, -2.5, 5.0]
)
print(f"Invalid data: {valid}")
for err in errors:
    print(f"  - {err}")

<details>
<summary>💡 Click to reveal solution</summary>

```python
def validate_kinematics(pt, eta, phi):
    """
    Validate particle kinematic values.
    """
    errors = []
    
    pt = np.asarray(pt)
    eta = np.asarray(eta)
    phi = np.asarray(phi)
    
    # Check pT
    if np.any(pt < 0):
        n_bad = np.sum(pt < 0)
        errors.append(f"pT must be >= 0: found {n_bad} negative values")
    
    # Check eta
    if np.any(np.abs(eta) > 5):
        n_bad = np.sum(np.abs(eta) > 5)
        errors.append(f"|eta| must be < 5: found {n_bad} out-of-range values")
    
    # Check phi
    if np.any(np.abs(phi) > np.pi):
        n_bad = np.sum(np.abs(phi) > np.pi)
        errors.append(f"phi must be in [-π, π]: found {n_bad} out-of-range values")
    
    # Check for NaN
    for name, arr in [('pT', pt), ('eta', eta), ('phi', phi)]:
        if np.any(np.isnan(arr)):
            errors.append(f"{name} contains NaN values")
    
    return len(errors) == 0, errors
```

</details>

### Advanced Version: Custom Exceptions and Error Recovery

In [ ]:
# TODO: Create custom exception classes for analysis errors

class AnalysisError(Exception):
    """Base exception for analysis errors."""
    pass

class InvalidEventError(AnalysisError):
    """Raised when event data is invalid."""
    # YOUR CODE HERE: Add event_id attribute
    pass

class PhysicsConstraintError(AnalysisError):
    """Raised when physics constraints are violated."""
    pass

class DataQualityError(AnalysisError):
    """Raised when data quality is unacceptable."""
    pass

<details>
<summary>💡 Click to reveal solution</summary>

```python
class AnalysisError(Exception):
    """Base exception for analysis errors."""
    pass

class InvalidEventError(AnalysisError):
    """Raised when event data is invalid."""
    def __init__(self, event_id, message):
        self.event_id = event_id
        self.message = message
        super().__init__(f"Event {event_id}: {message}")

class PhysicsConstraintError(AnalysisError):
    """Raised when physics constraints are violated."""
    def __init__(self, constraint, value, expected):
        self.constraint = constraint
        self.value = value
        self.expected = expected
        super().__init__(f"{constraint}: got {value}, expected {expected}")

class DataQualityError(AnalysisError):
    """Raised when data quality is unacceptable."""
    def __init__(self, metric, value, threshold):
        self.metric = metric
        self.value = value
        self.threshold = threshold
        super().__init__(f"Data quality issue: {metric}={value:.2f} (threshold: {threshold})")
```

</details>

In [ ]:
# TODO: Implement a robust event processor with error recovery

def process_event(event):
    """
    Process a single event with comprehensive error handling.
    
    Parameters:
    -----------
    event : dict
        Event data with keys: event_id, mu1_pt, mu1_eta, mu2_pt, mu2_eta, etc.
    
    Returns:
    --------
    dict : Processed event with invariant mass
    
    Raises:
    -------
    InvalidEventError : if event data is invalid
    PhysicsConstraintError : if physics constraints are violated
    """
    # YOUR CODE HERE
    # 1. Check required fields exist
    # 2. Validate kinematics
    # 3. Calculate invariant mass
    # 4. Check physics constraints (mass > 0, mass < 1000 GeV)
    
    pass

def process_events_with_recovery(events):
    """
    Process multiple events, recovering from individual failures.
    
    Parameters:
    -----------
    events : list of dict
        List of event data
    
    Returns:
    --------
    list : Successfully processed events
    list : Failed events with error info
    """
    successful = []
    failed = []
    
    # YOUR CODE HERE
    # Loop through events, catching errors and continuing
    
    return successful, failed

# Test data with some problematic events
test_events = [
    {'event_id': 1, 'mu1_pt': 30, 'mu1_eta': 0.5, 'mu1_phi': 0.1,
     'mu2_pt': 25, 'mu2_eta': -0.3, 'mu2_phi': 2.5},
    {'event_id': 2, 'mu1_pt': -10, 'mu1_eta': 0.5, 'mu1_phi': 0.1,  # Negative pT
     'mu2_pt': 25, 'mu2_eta': -0.3, 'mu2_phi': 2.5},
    {'event_id': 3, 'mu1_pt': 45, 'mu1_eta': 1.2, 'mu1_phi': -1.0,
     'mu2_pt': 40, 'mu2_eta': -1.5, 'mu2_phi': 1.5},
    {'event_id': 4},  # Missing data
]

successful, failed = process_events_with_recovery(test_events)
print(f"\nProcessed: {len(successful)} successful, {len(failed)} failed")

<details>
<summary>💡 Click to reveal solution</summary>

```python
def process_event(event):
    """
    Process a single event with comprehensive error handling.
    """
    event_id = event.get('event_id', 'unknown')
    
    # Check required fields
    required = ['mu1_pt', 'mu1_eta', 'mu1_phi', 'mu2_pt', 'mu2_eta', 'mu2_phi']
    missing = [f for f in required if f not in event]
    if missing:
        raise InvalidEventError(event_id, f"Missing fields: {missing}")
    
    # Validate kinematics
    if event['mu1_pt'] < 0 or event['mu2_pt'] < 0:
        raise InvalidEventError(event_id, "Negative pT values")
    
    if abs(event['mu1_eta']) > 5 or abs(event['mu2_eta']) > 5:
        raise InvalidEventError(event_id, "Eta out of range")
    
    # Calculate invariant mass
    deta = event['mu1_eta'] - event['mu2_eta']
    dphi = event['mu1_phi'] - event['mu2_phi']
    m2 = 2 * event['mu1_pt'] * event['mu2_pt'] * (np.cosh(deta) - np.cos(dphi))
    
    if m2 < 0:
        raise PhysicsConstraintError("m²", m2, ">= 0")
    
    mass = np.sqrt(m2)
    
    if mass > 1000:
        raise PhysicsConstraintError("mass", mass, "< 1000 GeV")
    
    return {**event, 'mass': mass}

def process_events_with_recovery(events):
    """
    Process multiple events, recovering from individual failures.
    """
    successful = []
    failed = []
    
    for event in events:
        try:
            result = process_event(event)
            successful.append(result)
        except InvalidEventError as e:
            failed.append({'event': event, 'error_type': 'InvalidEvent', 'message': str(e)})
        except PhysicsConstraintError as e:
            failed.append({'event': event, 'error_type': 'PhysicsConstraint', 'message': str(e)})
        except Exception as e:
            failed.append({'event': event, 'error_type': 'Unknown', 'message': str(e)})
    
    print(f"Processed {len(successful)} events successfully")
    if failed:
        print(f"Failed events:")
        for f in failed:
            print(f"  Event {f['event'].get('event_id', '?')}: {f['error_type']} - {f['message']}")
    
    return successful, failed
```

</details>

---
## Exercise 3.2: Testing Scientific Code (45 min)

### Physics Context
Testing ensures your analysis produces correct results. In particle physics, we test against known values (particle masses), physical constraints (energy conservation), and expected behavior.

In [ ]:
# First, let's define functions to test

def calculate_invariant_mass(pt1, eta1, phi1, pt2, eta2, phi2):
    """Calculate invariant mass of a particle pair."""
    deta = eta1 - eta2
    dphi = phi1 - phi2
    m2 = 2 * pt1 * pt2 * (np.cosh(deta) - np.cos(dphi))
    return np.sqrt(m2) if m2 >= 0 else 0

def apply_pt_cut(pt_values, threshold):
    """Apply pT threshold cut."""
    return pt_values[pt_values > threshold]

def calculate_delta_r(eta1, phi1, eta2, phi2):
    """Calculate angular separation ΔR."""
    deta = eta1 - eta2
    dphi = phi1 - phi2
    # Handle phi wrap-around
    if dphi > np.pi:
        dphi -= 2 * np.pi
    elif dphi < -np.pi:
        dphi += 2 * np.pi
    return np.sqrt(deta**2 + dphi**2)

### Beginner Version: Basic Tests

In [ ]:
# TODO: Write test functions using assert statements
# In Jupyter, we'll run tests manually (in real projects, use pytest)

def test_invariant_mass_known_value():
    """
    Test mass calculation with Z boson-like kinematics.
    Two back-to-back muons should give ~91 GeV.
    """
    # YOUR CODE HERE
    # Create two muons with opposite phi, same pT
    # that should give approximately Z mass
    
    mass = calculate_invariant_mass(
        pt1=45.6, eta1=0, phi1=0,
        pt2=45.6, eta2=0, phi2=np.pi
    )
    
    # Test that mass is close to expected value
    # YOUR CODE HERE: use assert with a tolerance
    
    print(f"✓ test_invariant_mass_known_value passed (mass={mass:.2f} GeV)")

def test_invariant_mass_positive():
    """Test that mass is always non-negative."""
    # YOUR CODE HERE
    # Test with various inputs
    
    print("✓ test_invariant_mass_positive passed")

def test_pt_cut_reduces_events():
    """Test that pT cut reduces number of events."""
    # YOUR CODE HERE
    
    print("✓ test_pt_cut_reduces_events passed")

def test_pt_cut_correct_values():
    """Test that only values above threshold remain."""
    # YOUR CODE HERE
    
    print("✓ test_pt_cut_correct_values passed")

# Run all tests
print("Running tests...\n")
test_invariant_mass_known_value()
test_invariant_mass_positive()
test_pt_cut_reduces_events()
test_pt_cut_correct_values()
print("\n✓ All tests passed!")

<details>
<summary>💡 Click to reveal solution</summary>

```python
def test_invariant_mass_known_value():
    """
    Test mass calculation with Z boson-like kinematics.
    """
    mass = calculate_invariant_mass(
        pt1=45.6, eta1=0, phi1=0,
        pt2=45.6, eta2=0, phi2=np.pi
    )
    
    # Z boson mass is ~91.2 GeV
    expected = 91.2
    tolerance = 1.0  # 1 GeV tolerance
    
    assert abs(mass - expected) < tolerance, f"Expected ~{expected}, got {mass}"
    print(f"✓ test_invariant_mass_known_value passed (mass={mass:.2f} GeV)")

def test_invariant_mass_positive():
    """Test that mass is always non-negative."""
    test_cases = [
        (30, 0.5, 0.1, 25, -0.3, 2.5),
        (100, 2.0, -1.5, 50, -1.0, 1.0),
        (10, 0, 0, 10, 0, np.pi),
    ]
    
    for pt1, eta1, phi1, pt2, eta2, phi2 in test_cases:
        mass = calculate_invariant_mass(pt1, eta1, phi1, pt2, eta2, phi2)
        assert mass >= 0, f"Mass should be non-negative, got {mass}"
    
    print("✓ test_invariant_mass_positive passed")

def test_pt_cut_reduces_events():
    """Test that pT cut reduces number of events."""
    pt_values = np.array([10, 20, 30, 40, 50])
    threshold = 25
    
    result = apply_pt_cut(pt_values, threshold)
    
    assert len(result) < len(pt_values), "Cut should reduce events"
    assert len(result) == 3, f"Expected 3 events, got {len(result)}"
    
    print("✓ test_pt_cut_reduces_events passed")

def test_pt_cut_correct_values():
    """Test that only values above threshold remain."""
    pt_values = np.array([10, 20, 30, 40, 50])
    threshold = 25
    
    result = apply_pt_cut(pt_values, threshold)
    
    assert all(result > threshold), "All values should be above threshold"
    assert list(result) == [30, 40, 50], f"Wrong values: {result}"
    
    print("✓ test_pt_cut_correct_values passed")
```

</details>

In [ ]:
# TODO: Test ΔR calculation

def test_delta_r_same_point():
    """ΔR between same point should be 0."""
    # YOUR CODE HERE
    pass

def test_delta_r_known_values():
    """Test with known geometric values."""
    # YOUR CODE HERE
    # ΔR for Δη=1, Δφ=0 should be 1
    # ΔR for Δη=0, Δφ=1 should be 1
    # ΔR for Δη=1, Δφ=1 should be sqrt(2)
    pass

def test_delta_r_phi_wraparound():
    """Test that phi wrap-around is handled correctly."""
    # YOUR CODE HERE
    # phi=3 and phi=-3 are close (not 6 apart)
    pass

# Run tests
test_delta_r_same_point()
test_delta_r_known_values()
test_delta_r_phi_wraparound()
print("\n✓ All ΔR tests passed!")

<details>
<summary>💡 Click to reveal solution</summary>

```python
def test_delta_r_same_point():
    """ΔR between same point should be 0."""
    dr = calculate_delta_r(0, 0, 0, 0)
    assert dr == 0, f"Expected 0, got {dr}"
    print("✓ test_delta_r_same_point passed")

def test_delta_r_known_values():
    """Test with known geometric values."""
    # Only eta difference
    dr1 = calculate_delta_r(1, 0, 0, 0)
    assert abs(dr1 - 1.0) < 1e-10, f"Expected 1.0, got {dr1}"
    
    # Only phi difference
    dr2 = calculate_delta_r(0, 1, 0, 0)
    assert abs(dr2 - 1.0) < 1e-10, f"Expected 1.0, got {dr2}"
    
    # Both (Pythagorean)
    dr3 = calculate_delta_r(1, 1, 0, 0)
    assert abs(dr3 - np.sqrt(2)) < 1e-10, f"Expected sqrt(2), got {dr3}"
    
    print("✓ test_delta_r_known_values passed")

def test_delta_r_phi_wraparound():
    """Test that phi wrap-around is handled correctly."""
    # phi=3 and phi=-3 are about 2π-6 ≈ 0.28 apart (wrapped)
    dr = calculate_delta_r(0, 3.0, 0, -3.0)
    expected = 2*np.pi - 6  # Wrapped distance
    
    assert abs(dr - expected) < 0.01, f"Expected ~{expected:.2f}, got {dr:.2f}"
    
    # Should be less than the naive |3 - (-3)| = 6
    assert dr < 1.0, "Wrapped ΔR should be small"
    
    print("✓ test_delta_r_phi_wraparound passed")
```

</details>

### Advanced Version: Parametrized Tests and Edge Cases

In [ ]:
# TODO: Create parametrized tests (testing same function with multiple inputs)

def parametrized_test_invariant_mass():
    """
    Test invariant mass calculation with multiple known particle pairs.
    """
    test_cases = [
        # (pt1, eta1, phi1, pt2, eta2, phi2, expected_mass, tolerance, description)
        (45.6, 0, 0, 45.6, 0, np.pi, 91.2, 1.0, "Z boson"),
        (1.5, 0, 0, 1.5, 0, np.pi, 3.0, 0.1, "J/psi-like"),
        # YOUR CODE HERE: Add more test cases
    ]
    
    for pt1, eta1, phi1, pt2, eta2, phi2, expected, tol, desc in test_cases:
        mass = calculate_invariant_mass(pt1, eta1, phi1, pt2, eta2, phi2)
        assert abs(mass - expected) < tol, f"{desc}: expected {expected}, got {mass}"
        print(f"✓ {desc}: mass={mass:.2f} GeV (expected ~{expected})")

parametrized_test_invariant_mass()

<details>
<summary>💡 Click to reveal solution</summary>

```python
def parametrized_test_invariant_mass():
    """
    Test invariant mass calculation with multiple known particle pairs.
    """
    test_cases = [
        # (pt1, eta1, phi1, pt2, eta2, phi2, expected_mass, tolerance, description)
        (45.6, 0, 0, 45.6, 0, np.pi, 91.2, 1.0, "Z boson"),
        (1.5, 0, 0, 1.5, 0, np.pi, 3.0, 0.1, "J/psi-like"),
        (20, 0, 0, 20, 0, 0, 0, 0.1, "Collinear (mass≈0)"),
        (50, 1.0, 0.5, 40, -0.5, 2.5, None, None, "Random kinematics"),  # Just check positive
    ]
    
    for pt1, eta1, phi1, pt2, eta2, phi2, expected, tol, desc in test_cases:
        mass = calculate_invariant_mass(pt1, eta1, phi1, pt2, eta2, phi2)
        
        if expected is not None:
            assert abs(mass - expected) < tol, f"{desc}: expected {expected}, got {mass}"
            print(f"✓ {desc}: mass={mass:.2f} GeV (expected ~{expected})")
        else:
            assert mass >= 0, f"{desc}: mass should be positive"
            print(f"✓ {desc}: mass={mass:.2f} GeV (verified positive)")
```

</details>

In [ ]:
# TODO: Test numerical precision and edge cases

def test_numerical_precision():
    """
    Test that calculations are numerically stable.
    """
    # YOUR CODE HERE
    # Test with very large values
    # Test with very small values
    # Test with values that might cause numerical issues
    
    # Example: very high pT
    mass_high_pt = calculate_invariant_mass(1000, 0, 0, 1000, 0, np.pi)
    assert np.isfinite(mass_high_pt), "Result should be finite"
    
    # YOUR CODE HERE: Add more tests
    
    print("✓ Numerical precision tests passed")

def test_edge_cases():
    """
    Test boundary conditions and edge cases.
    """
    # YOUR CODE HERE
    # Zero pT
    # Very small angles
    # phi at ±π
    
    print("✓ Edge case tests passed")

test_numerical_precision()
test_edge_cases()

<details>
<summary>💡 Click to reveal solution</summary>

```python
def test_numerical_precision():
    """
    Test that calculations are numerically stable.
    """
    # Very high pT (TeV scale)
    mass_high_pt = calculate_invariant_mass(1000, 0, 0, 1000, 0, np.pi)
    assert np.isfinite(mass_high_pt), "High pT should give finite result"
    assert mass_high_pt > 1900, "High pT back-to-back should give ~2*pT"
    
    # Very small pT (MeV scale in GeV)
    mass_low_pt = calculate_invariant_mass(0.001, 0, 0, 0.001, 0, np.pi)
    assert np.isfinite(mass_low_pt), "Low pT should give finite result"
    assert mass_low_pt >= 0, "Mass should be non-negative"
    
    # Very forward (large eta)
    mass_forward = calculate_invariant_mass(50, 4.5, 0, 50, -4.5, np.pi)
    assert np.isfinite(mass_forward), "Forward particles should give finite result"
    
    # Check consistency: same input should give same output
    m1 = calculate_invariant_mass(30, 0.5, 0.1, 25, -0.3, 2.5)
    m2 = calculate_invariant_mass(30, 0.5, 0.1, 25, -0.3, 2.5)
    assert m1 == m2, "Same input should give same output"
    
    print("✓ Numerical precision tests passed")

def test_edge_cases():
    """
    Test boundary conditions and edge cases.
    """
    # Zero pT (should give zero mass)
    mass_zero = calculate_invariant_mass(0, 0, 0, 0, 0, 0)
    assert mass_zero == 0, f"Zero pT should give zero mass, got {mass_zero}"
    
    # One particle has zero pT
    mass_one_zero = calculate_invariant_mass(50, 0, 0, 0, 0, 0)
    assert mass_one_zero == 0, "One zero pT should give zero mass"
    
    # phi at boundaries
    mass_phi_pi = calculate_invariant_mass(50, 0, np.pi, 50, 0, -np.pi)
    assert np.isfinite(mass_phi_pi), "phi at ±π should work"
    # These are actually the same point, so mass should be 0
    assert mass_phi_pi < 1, f"Same phi should give ~0 mass, got {mass_phi_pi}"
    
    print("✓ Edge case tests passed")
```

</details>

---
## Exercise 3.3: Debugging (30 min)

### Physics Context
Debug the following analysis code that contains intentional bugs. Find and fix them!

### Beginner Version: Find the Bugs

In [ ]:
# This code has BUGS! Find and fix them.

def calculate_transverse_momentum_BUGGY(px, py):
    """Calculate pT from px and py components."""
    # BUG: Wrong formula
    return px + py

def apply_eta_cut_BUGGY(particles, eta_max):
    """Select particles with |eta| < eta_max."""
    # BUG: Wrong comparison
    return particles[particles['eta'] < eta_max]

def calculate_mean_pt_BUGGY(pt_values):
    """Calculate mean pT."""
    # BUG: Off-by-one error
    total = 0
    for i in range(1, len(pt_values)):
        total += pt_values[i]
    return total / len(pt_values)

# Test data
test_px = np.array([3, 4, 0])
test_py = np.array([4, 3, 5])

particles_df = pd.DataFrame({
    'eta': [-3.0, -1.0, 0.5, 1.5, 2.5],
    'pt': [10, 20, 30, 40, 50]
})

pt_values = np.array([10, 20, 30, 40, 50])

# Run buggy code
print("=== BUGGY CODE OUTPUT ===")
print(f"pT values: {calculate_transverse_momentum_BUGGY(test_px, test_py)}")
print(f"Expected: [5, 5, 5]")

selected = apply_eta_cut_BUGGY(particles_df, 2.0)
print(f"\nSelected particles (|eta| < 2.0): {len(selected)}")
print(f"Expected: 3 particles")

mean_pt = calculate_mean_pt_BUGGY(pt_values)
print(f"\nMean pT: {mean_pt}")
print(f"Expected: 30")

In [ ]:
# TODO: Fix the bugs!

def calculate_transverse_momentum_FIXED(px, py):
    """Calculate pT from px and py components."""
    # YOUR CODE HERE
    pass

def apply_eta_cut_FIXED(particles, eta_max):
    """Select particles with |eta| < eta_max."""
    # YOUR CODE HERE
    pass

def calculate_mean_pt_FIXED(pt_values):
    """Calculate mean pT."""
    # YOUR CODE HERE
    pass

# Test fixed code
print("=== FIXED CODE OUTPUT ===")
print(f"pT values: {calculate_transverse_momentum_FIXED(test_px, test_py)}")

selected = apply_eta_cut_FIXED(particles_df, 2.0)
print(f"\nSelected particles (|eta| < 2.0): {len(selected)}")

mean_pt = calculate_mean_pt_FIXED(pt_values)
print(f"\nMean pT: {mean_pt}")

<details>
<summary>💡 Click to reveal solution</summary>

```python
def calculate_transverse_momentum_FIXED(px, py):
    """Calculate pT from px and py components."""
    # FIX: Use Pythagorean theorem, not addition
    return np.sqrt(px**2 + py**2)

def apply_eta_cut_FIXED(particles, eta_max):
    """Select particles with |eta| < eta_max."""
    # FIX: Need absolute value for |eta|
    return particles[np.abs(particles['eta']) < eta_max]

def calculate_mean_pt_FIXED(pt_values):
    """Calculate mean pT."""
    # FIX 1: Start from index 0, not 1
    # FIX 2: Or just use np.mean()
    return np.mean(pt_values)
    
    # Alternative manual fix:
    # total = 0
    # for i in range(len(pt_values)):  # Start from 0
    #     total += pt_values[i]
    # return total / len(pt_values)
```

</details>

### Advanced Version: Debug Complex Analysis

In [ ]:
# This analysis code has multiple bugs. Find them all!

def analyze_dimuon_events_BUGGY(events):
    """
    Analyze dimuon events and find Z candidates.
    
    Expected behavior:
    1. Apply pT > 20 GeV cut on both muons
    2. Apply |eta| < 2.4 cut on both muons  
    3. Calculate invariant mass
    4. Select events with 70 < mass < 110 GeV
    5. Return selected events and mass distribution
    """
    selected_events = []
    masses = []
    
    for event in events:
        # BUG 1: Wrong pT cut logic
        if event['mu1_pt'] > 20 or event['mu2_pt'] > 20:
            continue
        
        # BUG 2: Missing absolute value
        if event['mu1_eta'] > 2.4 or event['mu2_eta'] > 2.4:
            continue
        
        # BUG 3: Wrong mass formula (missing factor of 2)
        deta = event['mu1_eta'] - event['mu2_eta']
        dphi = event['mu1_phi'] - event['mu2_phi']
        m2 = event['mu1_pt'] * event['mu2_pt'] * (np.cosh(deta) - np.cos(dphi))
        mass = np.sqrt(m2) if m2 > 0 else 0
        
        # BUG 4: Wrong mass window comparison
        if mass < 70 and mass > 110:
            continue
        
        selected_events.append(event)
        masses.append(mass)
    
    # BUG 5: Wrong statistics calculation
    if masses:
        mean_mass = sum(masses) / (len(masses) - 1)  # Off by one
    else:
        mean_mass = 0
    
    return selected_events, masses, mean_mass

# Test data - should have Z candidates
test_events = [
    {'mu1_pt': 45, 'mu1_eta': 0.5, 'mu1_phi': 0.1,
     'mu2_pt': 40, 'mu2_eta': -0.3, 'mu2_phi': np.pi + 0.1},  # Should be Z
    {'mu1_pt': 30, 'mu1_eta': 1.0, 'mu1_phi': 0.5,
     'mu2_pt': 35, 'mu2_eta': -1.2, 'mu2_phi': -2.5},  # Should be Z
    {'mu1_pt': 10, 'mu1_eta': 0.5, 'mu1_phi': 0.1,
     'mu2_pt': 40, 'mu2_eta': -0.3, 'mu2_phi': 3.0},  # Fails pT cut
    {'mu1_pt': 45, 'mu1_eta': 3.0, 'mu1_phi': 0.1,
     'mu2_pt': 40, 'mu2_eta': -0.3, 'mu2_phi': 3.0},  # Fails eta cut
]

selected, masses, mean_mass = analyze_dimuon_events_BUGGY(test_events)
print(f"Selected events: {len(selected)}")
print(f"Expected: 2 events")
print(f"\nMasses: {[f'{m:.1f}' for m in masses]}")
print(f"Mean mass: {mean_mass:.1f} GeV")
print(f"Expected mean: ~91 GeV")

In [ ]:
# TODO: Fix ALL the bugs!

def analyze_dimuon_events_FIXED(events):
    """
    Analyze dimuon events and find Z candidates.
    """
    selected_events = []
    masses = []
    
    for event in events:
        # YOUR CODE HERE: Fix all 5 bugs
        pass
    
    return selected_events, masses, mean_mass

# Test fixed code
selected, masses, mean_mass = analyze_dimuon_events_FIXED(test_events)
print(f"Selected events: {len(selected)}")
print(f"Masses: {[f'{m:.1f}' for m in masses]}")
print(f"Mean mass: {mean_mass:.1f} GeV")

<details>
<summary>💡 Click to reveal solution</summary>

```python
def analyze_dimuon_events_FIXED(events):
    """
    Analyze dimuon events and find Z candidates.
    """
    selected_events = []
    masses = []
    
    for event in events:
        # FIX 1: Need AND (both muons pass), not OR
        # Also: continue skips, so need to negate the condition
        if not (event['mu1_pt'] > 20 and event['mu2_pt'] > 20):
            continue
        
        # FIX 2: Add absolute value for eta cut
        if abs(event['mu1_eta']) > 2.4 or abs(event['mu2_eta']) > 2.4:
            continue
        
        # FIX 3: Add factor of 2 in mass formula
        deta = event['mu1_eta'] - event['mu2_eta']
        dphi = event['mu1_phi'] - event['mu2_phi']
        m2 = 2 * event['mu1_pt'] * event['mu2_pt'] * (np.cosh(deta) - np.cos(dphi))
        mass = np.sqrt(m2) if m2 > 0 else 0
        
        # FIX 4: Change 'and' to 'or' (want to SKIP if OUTSIDE window)
        if mass < 70 or mass > 110:
            continue
        
        selected_events.append(event)
        masses.append(mass)
    
    # FIX 5: Divide by len(masses), not len(masses)-1
    if masses:
        mean_mass = sum(masses) / len(masses)
    else:
        mean_mass = 0
    
    return selected_events, masses, mean_mass
```

**Summary of bugs:**
1. `or` should be `and` for pT cut (need BOTH muons above threshold)
2. Missing `abs()` for eta cut (need |eta| < 2.4)
3. Missing factor of 2 in invariant mass formula
4. `and` should be `or` in mass window check (skip if OUTSIDE window)
5. Dividing by `len(masses) - 1` instead of `len(masses)`

</details>

---
## Summary

Today you learned:

✅ **Error Handling**: try/except blocks, custom exceptions, validation  
✅ **Testing**: Writing test functions, parametrized tests, edge cases  
✅ **Debugging**: Finding and fixing common bugs in analysis code  

**This afternoon:** Final Integration Project - Apply everything you've learned!

---

**Great work! 🎉**